<span style="color:#FF7A00; font-size:30px; font-weight:bold;">
BDP Term Project (1조)
</span>

<span style="font-size:20px; font-weight:bold;">
encar.com에서 국산 중고차 매물별 보험이력 불러오기
</span>

<span style="font-size:17px; font-weight:bold;">
Driver Load
</span>

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
import csv
import time

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

print("작업 완료")

작업 완료


### 크롤링 셋업
#### `MAX_FETCH_COUNT`
- 매물 리스트 조회 횟수 조절
- `count` `1` 당 `250` 개의 매물 `id` 와 `price` 데이터를 가져옴
- 대략 `650` 정도 값이면 국산차 매물 데이터를 전부 가져올 수 있음
#### `OUTPUT_FILE_PATH`
- 파일 저장 경로를 설정

In [9]:
MAX_FETCH_COUNT = 650
OUTPUT_FILE_PATH = "./vehicles_price.csv"

### crawling

In [10]:
def main():
    base_url = "https://car.encar.com/list/car?page={page}&search=%7B%22action%22%3A%22(And.Hidden.N._.CarType.Y.)%22%2C%22sort%22%3A%22MobileModifiedDate%22%7D"
    vehicles = fetch_vehicle_ids(driver, base_url, MAX_FETCH_COUNT)
        
    output_file = OUTPUT_FILE_PATH
    try:
        with open(output_file, mode="w", newline="", encoding="utf-8") as file:
            writer = csv.writer(file)
            writer.writerow(["id", "price"])  # 헤더 작성
            for vehicle_id, price in vehicles.items():
                writer.writerow([vehicle_id, price])
        print(f"매물 데이터 {len(vehicles)}개가 {output_file} 파일에 저장되었습니다.")
    except Exception as e:
        print("CSV 저장 중 오류 발생:", e)

def fetch_vehicle_ids(driver, base_url, max_fetch_count = 1):
    vehicles = {}
    
    log_timing = 0
    for page in range(1, max_fetch_count + 1):
        log_timing += 1
        if(log_timing >= 20):
            print(f"{max_fetch_count}개 중 {page}번째 페이지 로드중")
            log_timing = 0

        driver.get(base_url.format(page = page))
    
        # 요소 팝업까지 대기 및 무결성 검사 
        try:
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.XPATH, "//div[@data-impression]")))
            elements = driver.find_elements(By.XPATH, "//div[@data-impression]")
        except Exception:
            print(f"페이지 {page} 로드 실패. 크롤링을 중단합니다.")
            break

        # 페이지에 데이터 추출 후 dict에 보관
        for e in elements:
            is_reserved = e.find_elements(By.XPATH, ".//span[contains(text(), '계약중')]")
            if is_reserved: 
                continue # '계약중'인 경우 건너뜀

            data_impression = e.get_attribute("data-impression")
            if data_impression:
                # data-impression 값을 분리하여 id와 price 추출
                parts = data_impression.split("|")
                if len(parts) >= 2:
                    vehicle_id = parts[0]  # ID
                    price = parts[1]       # 가격
                    # 중복되지 않게 저장
                    vehicles[vehicle_id] = price
    return vehicles

if __name__ == "__main__":
    main()

650개 중 20번째 페이지 로드중
650개 중 40번째 페이지 로드중
650개 중 60번째 페이지 로드중
650개 중 80번째 페이지 로드중
650개 중 100번째 페이지 로드중
650개 중 120번째 페이지 로드중
650개 중 140번째 페이지 로드중
650개 중 160번째 페이지 로드중
650개 중 180번째 페이지 로드중
650개 중 200번째 페이지 로드중
650개 중 220번째 페이지 로드중
650개 중 240번째 페이지 로드중
650개 중 260번째 페이지 로드중
650개 중 280번째 페이지 로드중
650개 중 300번째 페이지 로드중
650개 중 320번째 페이지 로드중
650개 중 340번째 페이지 로드중
650개 중 360번째 페이지 로드중
650개 중 380번째 페이지 로드중
650개 중 400번째 페이지 로드중
650개 중 420번째 페이지 로드중
650개 중 440번째 페이지 로드중
650개 중 460번째 페이지 로드중
650개 중 480번째 페이지 로드중
650개 중 500번째 페이지 로드중
650개 중 520번째 페이지 로드중
650개 중 540번째 페이지 로드중
650개 중 560번째 페이지 로드중
650개 중 580번째 페이지 로드중
650개 중 600번째 페이지 로드중
650개 중 620번째 페이지 로드중
페이지 626 로드 실패. 크롤링을 중단합니다.
매물 데이터 110532개가 ./vehicles_price.csv 파일에 저장되었습니다.


In [11]:
try:
    driver.close()
except:
    print("driver 연결 오류")